# Horse or Human Classification

Model klasifikasi gambar untuk membedakan antara kuda dan manusia menggunakan TensorFlow dan CNN.

In [3]:
import urllib.request
import zipfile
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'tensorflow'

## Unduh dan Ekstrak Dataset

In [ ]:
def solution_05():
    # Download training dataset
    train_url = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
    urllib.request.urlretrieve(train_url, 'horse-or-human.zip')

    with zipfile.ZipFile('horse-or-human.zip', 'r') as zip_ref:
        zip_ref.extractall('data/horse-or-human')

    # Download validation dataset
    val_url = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
    urllib.request.urlretrieve(val_url, 'validation-horse-or-human.zip')

    with zipfile.ZipFile('validation-horse-or-human.zip', 'r') as zip_ref:
        zip_ref.extractall('data/validation-horse-or-human')
    
    TRAINING_DIR = 'data/horse-or-human'
    VALIDATION_DIR = 'data/validation-horse-or-human'

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
    )

    validation_generator = validation_datagen.flow_from_directory(
        VALIDATION_DIR,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
    )

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # DO NOT CHANGE THIS LINE!
    ])

    model.compile(loss='binary_crossentropy',
                optimizer=RMSprop(learning_rate=0.001),
                metrics=['accuracy'])

    model.summary()

    class StopTrainingCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            acc = logs.get('accuracy')
            val_acc = logs.get('val_accuracy')
            if acc and val_acc and acc > 0.83 and val_acc > 0.83:
                print(f"\nStopping training: accuracy={acc*100:.2f}%, val_accuracy={val_acc*100:.2f}% > 83%")
                self.model.stop_training = True

    early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    stop_callback = StopTrainingCallback()

    history = model.fit(
        train_generator,
        epochs=20,
        validation_data=validation_generator,
        callbacks=[early_stop, stop_callback],
        verbose=1
    )

    train_acc = history.history['accuracy'][-1]
    val_acc = history.history['val_accuracy'][-1]

    print(f"Training Accuracy: {train_acc*100:.2f}%")
    print(f"Validation Accuracy: {val_acc*100:.2f}%")

## Melatih Model

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.5056 - loss: 2.5076 - val_accuracy: 0.8594 - val_loss: 0.5091
Epoch 2/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.7255 - loss: 0.5302 - val_accuracy: 0.7539 - val_loss: 0.9447
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 33s 993ms/step - accuracy: 0.8189 - loss: 0.4019 - val_accuracy: 0.6211 - val_loss: 1.8142
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 948ms/step - accuracy: 0.8760 - loss: 0.2743
Stopping training: accuracy=88.41%, val_accuracy=84.77% > 83%
33/33 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.8762 - loss: 0.2743 - val_accuracy: 0.8477 - val_loss: 0.3841
Training Accuracy: 88.41%
Validation Accuracy: 84.77%


## Menyimpan Model

In [ ]:
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_05()
    model.save("model_05.h5")